In [58]:
import os
import pandas as pd
from tqdm import tqdm
import pickle


In [59]:
os.chdir("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/")
filename = "europarl-v7.da-en.da"
with open(filename, "r", encoding="UTF-8") as file:
    lines = file.readlines()
with open("nutids_r.pickle", "rb") as f:
    nutids_r = pickle.load(f)
with open("nutids_r_stem.pickle", "rb") as f:
    nutids_r_stem = pickle.load(f)

In [60]:
step_rate = 5
pre_pad_result = []
for i in tqdm(range(0, len(lines),step_rate)):
    pre_pad_result.append((' '.join(lines[i:i+step_rate])).split())

100%|██████████| 393760/393760 [00:09<00:00, 39520.11it/s] 


In [61]:
scope = 10
middle = int(scope/2-1)
padding = int(scope/2-1)

In [62]:
pre_cleaned_result = []
for i in tqdm(range(len(pre_pad_result))):
    lst = pre_pad_result[i]
    lst = ["<PAD>"]*padding + lst + ["<PAD>"]*padding
    pre_cleaned_result.append(lst)

100%|██████████| 393760/393760 [00:22<00:00, 17325.81it/s] 


In [63]:
post_pad_result = []
char = "*@;:!\"?«».,"

for i in tqdm(range(len(pre_cleaned_result))):
    pre_cleaned_lst = pre_cleaned_result[i]
    lst = [word.translate(str.maketrans('', '', ''.join(char))) for word in pre_cleaned_lst]
    post_pad_result.append(lst)

100%|██████████| 393760/393760 [01:10<00:00, 5578.96it/s]


In [64]:
saved_post_pad_result = post_pad_result

In [65]:
post_pad_result = saved_post_pad_result[:15000]

In [66]:
bøjning = {0: "infinitiv", 1: "førnutidsformen"}
half_scope = int(scope/2)
big_lst = []
output_lst = []


for i in tqdm(range(len(post_pad_result))):
    current_lst = post_pad_result[i]
    for x in range(len(current_lst)):
        current_word = current_lst[x]
        try: stemmed_word = nutids_r_stem[current_word]
        except: continue
        current_dataset = current_lst[x-half_scope:x+half_scope+1]
        current_dataset[half_scope] = stemmed_word
        big_lst.append(" ".join(current_dataset))
        if nutids_r[stemmed_word][0] == current_word:
            output = 0
        elif nutids_r[stemmed_word][1] == current_word:
            output = 1
        else:
            print("ERROR")
        output_lst.append(output)


100%|██████████| 15000/15000 [00:01<00:00, 14782.07it/s]


In [67]:
df = pd.DataFrame()
df["comment_text"] = big_lst
df["label"] = output_lst

print(len(df))
df = df[:6000000]

def distribution(df):
    print(df["label"].value_counts())

distribution(df)
print(len(df))
header = ["comment_text", "label"]
df.to_csv("../Datasets/EuroparlNutidsr.csv", encoding="UTF-8", index=False, sep=";")

217020
0    113611
1    103409
Name: label, dtype: int64
217020
